# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [87]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
import string
import numpy as np

In [88]:
nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [89]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [90]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [91]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('Labelled_table', engine)


In [92]:
df.head(4)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [93]:
df.tail(4)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
26212,30262,A suitable candidate has been selected and OCH...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,30264,"Some 2,000 women protesting against the conduc...",None,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
26215,30265,A radical shift in thinking came about as a re...,None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [94]:
df.groupby("related").count()

,id,message,original,genre,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,,,,,,,,,,,,,,,,,,,,,
0,6122,6122,3395,6122,6122,6122,6122,6122,6122,6122,...,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122
1,19906,19906,6643,19906,19906,19906,19906,19906,19906,19906,...,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906
2,188,188,132,188,188,188,188,188,188,188,...,188,188,188,188,188,188,188,188,188,188


In [95]:
# Given value 2 in the related field are neglible so it could be error. Replacing 2 with 1 to consider it a valid response.
# Alternatively, we could have assumed it to be 0 also. In the absence of information I have gone with majority class.
df['related']=df['related'].map(lambda x: 1 if x == 2 else x)

In [96]:
df.groupby("related").count()

,id,message,original,genre,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,,,,,,,,,,,,,,,,,,,,,
0,6122,6122,3395,6122,6122,6122,6122,6122,6122,6122,...,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122
1,20094,20094,6775,20094,20094,20094,20094,20094,20094,20094,...,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094


In [97]:
df.groupby("aid_related").count()

,id,message,original,genre,related,request,offer,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
aid_related,,,,,,,,,,,,,,,,,,,,,
0,15356,15356,6232,15356,15356,15356,15356,15356,15356,15356,...,15356,15356,15356,15356,15356,15356,15356,15356,15356,15356
1,10860,10860,3938,10860,10860,10860,10860,10860,10860,10860,...,10860,10860,10860,10860,10860,10860,10860,10860,10860,10860


In [98]:
df['genre'].unique()

array(['direct', 'social', 'news'], dtype=object)

In [99]:
df['request'].unique()

array([0, 1])

In [100]:
df['offer'].unique()

array([0, 1])

In [101]:
df['child_alone'].unique()

array([0])

In [102]:
#Remove child alone as it has all zeros only
df = df.drop(['child_alone'],axis=1)

In [103]:
X = df['message']
y = df.drop(['message', 'genre', 'id', 'original'], axis=1)
category_names = y.columns.tolist()

### 2. Write a tokenization function to process your text data

In [104]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
import pickle

from scipy.stats import gmean
# import relevant functions/modules from the sklearn
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin

In [105]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [106]:
stop_words = nltk.corpus.stopwords.words("english")
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
remove_punc_table = str.maketrans('', '', string.punctuation)

In [107]:
def tokenize(text):
    """
    Tokenize the text function
    
    Arguments:
        text -> Text message which needs to be tokenized
    Output:
        clean_tokens -> List of tokens extracted from the provided text
    """
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [108]:
pipeline1 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))
            
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

pipeline2 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ])),

            ('starting_verb_transformer', StartingVerbExtractor())
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [109]:
from sklearn.model_selection import train_test_split

In [110]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fitted = pipeline1.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [111]:
from sklearn.metrics import classification_report

In [112]:
y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)



In [113]:
from sklearn.utils.multiclass import type_of_target
import numpy as np

In [114]:
y_test.values

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [115]:
y_prediction_test

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [116]:
classification_report(y_test.values, y_prediction_test, target_names=category_names)

'                        precision    recall  f1-score   support\n\n               related       0.82      0.95      0.88      5016\n               request       0.76      0.57      0.65      1113\n                 offer       0.00      0.00      0.00        29\n           aid_related       0.76      0.61      0.67      2722\n          medical_help       0.62      0.30      0.40       528\n      medical_products       0.62      0.30      0.40       329\n     search_and_rescue       0.55      0.20      0.29       177\n              security       0.44      0.07      0.12       110\n              military       0.63      0.34      0.44       214\n                 water       0.74      0.63      0.68       427\n                  food       0.80      0.75      0.77       699\n               shelter       0.77      0.57      0.65       596\n              clothing       0.62      0.37      0.46       104\n                 money       0.38      0.22      0.28       127\n        missing_people

In [117]:
print(classification_report(y_test.values, y_prediction_test, target_names=category_names))

                        precision    recall  f1-score   support

               related       0.82      0.95      0.88      5016
               request       0.76      0.57      0.65      1113
                 offer       0.00      0.00      0.00        29
           aid_related       0.76      0.61      0.67      2722
          medical_help       0.62      0.30      0.40       528
      medical_products       0.62      0.30      0.40       329
     search_and_rescue       0.55      0.20      0.29       177
              security       0.44      0.07      0.12       110
              military       0.63      0.34      0.44       214
                 water       0.74      0.63      0.68       427
                  food       0.80      0.75      0.77       699
               shelter       0.77      0.57      0.65       596
              clothing       0.62      0.37      0.46       104
                 money       0.38      0.22      0.28       127
        missing_people       0.54      

### 6. Improve your model
Use grid search to find better parameters. 

In [118]:
# pipeline1.get_params().keys()
parameters_grid = {'classifier__estimator__learning_rate': [0.01, 0.02, 0.05],
              'classifier__estimator__n_estimators': [10, 20, 40]}

#parameters_grid = {'classifier__estimator__n_estimators': [10, 20, 40]}

cv = GridSearchCV(pipeline1, param_grid=parameters_grid, scoring='f1_micro', n_jobs=-1)
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('count_vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'classifier__estimator__learning_rate': [0.01, 0.02, 0.05], 'classifier__estimator__n_estimators': [10, 20, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=0)

In [119]:
# Get the prediction values from the grid search cross validator
y_prediction_test = cv.predict(X_test)
y_prediction_train = cv.predict(X_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [120]:
# Print classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.77      1.00      0.87      5016
               request       0.60      0.39      0.48      1113
                 offer       0.00      0.00      0.00        29
           aid_related       0.80      0.19      0.30      2722
          medical_help       0.69      0.12      0.21       528
      medical_products       0.69      0.11      0.19       329
     search_and_rescue       0.63      0.20      0.31       177
              security       0.00      0.00      0.00       110
              military       0.50      0.01      0.02       214
                 water       0.57      0.83      0.68       427
                  food       0.77      0.69      0.73       699
               shelter       0.83      0.30      0.44       596
              clothing       0.61      0.26      0.36       104
                 money       0.00      0.00      0.00       127
        missing_people       0.59      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [121]:
# Print classification report on training data
print('\n',classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.77      1.00      0.87     15078
               request       0.61      0.38      0.47      3361
                 offer       1.00      0.02      0.04        89
           aid_related       0.80      0.19      0.31      8138
          medical_help       0.62      0.11      0.19      1556
      medical_products       0.74      0.13      0.23       984
     search_and_rescue       0.63      0.19      0.29       547
              security       0.00      0.00      0.00       361
              military       0.00      0.00      0.00       646
                 water       0.58      0.85      0.69      1245
                  food       0.77      0.67      0.72      2224
               shelter       0.85      0.30      0.44      1718
              clothing       0.79      0.34      0.47       301
                 money       0.00      0.00      0.00       477
        missing_people       0.71    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [122]:
#Use pipeline2 which includes StartingVerbEstimator
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fitted = pipeline2.fit(X_train, y_train)

y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)

# Print classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))


                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      5039
               request       0.74      0.52      0.62      1112
                 offer       0.00      0.00      0.00        34
           aid_related       0.75      0.59      0.66      2745
          medical_help       0.60      0.24      0.35       492
      medical_products       0.57      0.30      0.40       304
     search_and_rescue       0.51      0.17      0.26       185
              security       0.35      0.08      0.13       113
              military       0.55      0.32      0.41       208
                 water       0.74      0.64      0.69       380
                  food       0.81      0.66      0.73       715
               shelter       0.74      0.53      0.62       569
              clothing       0.64      0.41      0.50       103
                 money       0.54      0.26      0.35       159
        missing_people       0.67      

In [123]:
# Print classification report on training data
print('\n',classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.83      0.94      0.88     15055
               request       0.79      0.56      0.65      3362
                 offer       0.23      0.06      0.09        84
           aid_related       0.77      0.61      0.68      8115
          medical_help       0.65      0.28      0.39      1592
      medical_products       0.71      0.37      0.48      1009
     search_and_rescue       0.69      0.23      0.34       539
              security       0.38      0.06      0.11       358
              military       0.69      0.37      0.48       652
                 water       0.77      0.65      0.71      1292
                  food       0.82      0.69      0.75      2208
               shelter       0.79      0.54      0.64      1745
              clothing       0.76      0.46      0.57       302
                 money       0.66      0.34      0.45       445
        missing_people       0.67    

### 9. Export your model as a pickle file

In [124]:
m = pickle.dumps('classifier.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.